In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os.path as op
import numpy as np
import mne

mne.set_log_level('WARNING')

subjects_dir = op.expanduser("~/data/pic-name-data-bids/MRI/")
res_dir = op.expanduser("~/research/results/picname")

In [2]:
bili = [8, 9, 12]
# clil = [1, 6, 10, 11, 15]
mono = [3, 4, 5, 13, 14]
# subjects = ["sub-%02d" % i for i in (mono+bili)]
subjects = ["sub-%02d" % i for i in bili]
subjects

['sub-08', 'sub-09', 'sub-12']

In [3]:
dom = {'sub-01': 'L',
       'sub-02': 'L',
       'sub-03': 'R',
       'sub-04': 'R',
       'sub-05': 'R',
       'sub-06': 'R',
       'sub-08': 'R',
       'sub-09': 'R',
       'sub-10': 'R',
       'sub-11': 'R',
       'sub-12': 'R',
       'sub-13': 'L',
       'sub-14': 'R',
       'sub-15': 'R',
      }

In [4]:
for sub in subjects:
    print(sub, dom[sub])

sub-08 R
sub-09 R
sub-12 R


In [5]:
evokeds_list = []
for subject in subjects:
    fname = op.join(res_dir, subject, subject+'-ave.fif')
    ev = mne.read_evokeds(fname)
    evokeds_list.append(ev[0])

In [6]:
f, axes = plt.subplots(2, 2, figsize=(13, 9), sharex=True, sharey=True)
for ax, subject, evoked in zip(axes.ravel(), subjects, evokeds_list):
    evoked.plot(spatial_colors=True, gfp=True, proj=False, time_unit='ms', picks='mag',
                show=False, axes=ax, titles=subject) 
plt.tight_layout()

plt.savefig(op.join(res_dir, 'evoked_' + 'bili' + '.pdf'))

<ipython-input-6-2592b25f8c9f>:5: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
# f, axes = plt.subplots(2, 4, figsize=(13, 9), sharex=True, sharey=True)
# for ax, subject, evokeds in zip(axes.ravel(), subjects, evokeds_list):
#     mne.viz.plot_evoked(evokeds, picks='MEG2343', show=False, axes=ax, 
#                         titles=subject, spatial_colors=True, xlim=(-0.1,0.6))  
# plt.tight_layout()

# # plt.savefig(op.join(res_dir, 'evoked_all_subjects.pdf'))

In [7]:
evoked_gave = mne.grand_average(evokeds_list)
evoked_gave.plot(spatial_colors=True)

plt.savefig(op.join(res_dir, 'evoked_group_' + 'bili' + '.pdf'))

In [ ]:
# evkd = []
# for subject, evokeds in zip(subjects, evokeds_list):
#     mne.viz.plot_evoked(evokeds, picks='MEG2343', show=False, xlim=(-0.1,0.6))
# plt.show()

In [8]:
stc_fnames = []
for subject in subjects:
    if dom[subject]=="L":
        stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject + '-lh.stc'))
    elif dom[subject]=="R":
        stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject + '-rh.stc'))
    else:
        raise ValueError("Check $dom. It can only have 'L'/'R' as values.")
stcs = [mne.read_source_estimate(stc_fname) for stc_fname in stc_fnames]
average_stc = np.mean(stcs)

In [17]:
vertno_peak, t_peak = average_stc.get_peak()
print(t_peak)

surfer_kwargs = dict(subject="fsaverage", hemi='both', subjects_dir=subjects_dir,
        clim=dict(kind='value', lims=[2.5, 5, 6]), views='parietal',
        initial_time=t_peak, time_unit='s', size=(800, 800), smoothing_steps=10,
        time_viewer=False)

0.15700000000000003


In [20]:
brain = average_stc.plot(**surfer_kwargs)
brain.add_text(0.1, 0.9, 'dSPM (with location of maximal activation)', 'title',
              font_size=14)
brain.add_foci(vertno_peak, coords_as_verts=True, hemi='lh', color='blue',
              scale_factor=0.6, alpha=0.5)

# Draw figure and save image
dspm_fname = op.join(res_dir, "dspm_fsavg_bili_R.png")
# brain.save_image(dspm_fname)

# Generate and save movie
dspm_movie_fname = op.join(res_dir, 'dspm_movie_fsavg_bili_R.mov')
brain.save_movie(dspm_movie_fname, tmin=0.05, tmax=0.55, interpolation='linear',
                time_dilation=20, framerate=10, time_viewer=True)
brain.close()

In [ ]:
# # Estimate peak SNR and find corresponding time
# snr, _ = estimate_snr(evoked_gave, inverse_operator, verbose=True)
# nt_snr = np.argmax(snr)
# SNR = snr[nt_snr]
# print('\nMax SNR at %0.3f s : %0.3f' % (evoked.times[nt_snr], SNR))

In [ ]:
# labels = mne.read_labels_from_annot('fsaverage', 'HCPMMP1', 'both', subjects_dir=subjects_dir)
# labels_vis = []
# ROI = ['L_V1_ROI-lh', 'R_V1_ROI-rh']
# for lbl in labels:
#     if lbl.name in ROI:
#         labels_vis.append(lbl)
# label = labels_vis[0]
# for i in range(1, len(labels_vis)):
#     label = label + labels_vis[i]

# average_stc_label = average_stc.in_label(label)
# average_stc_label_mean = np.mean(average_stc_label.data, axis=0)

In [ ]:
# Evoked responses of required brain sources
# ROIs = ["LO", "Wernicke", "Broca"]
# evoked_roi = {}
# for roi in ROIs:
label = get_roi_labels(subject, subjects_dir, roi, par['dominant_brain'])
#     label_ts = mne.extract_label_time_course(stc, label, src, mode='mean_flip',
#                                             return_generator=True)
#     label_ts = label_ts.transpose()
#     evoked_roi[roi] = label_ts        

In [ ]:
average_stc.in_label

In [ ]:
times = 1e3 * average_stc.times # times in ms
plt.figure()
h0 = plt.plot(times, average_stc_label.data.T, 'k')
h1, = plt.plot(times, average_stc_label_mean, 'r', linewidth=3)
plt.show()

# plt.savefig(op.join(res_dir, 'evoked_label_group.pdf'))